In [1]:
from dotenv import load_dotenv
load_dotenv("project.env") 

True

In [2]:
import pandas as pd
import itertools

In [3]:
import os
import openai as oai

import langchain_openai as lcai
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, PromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.runnables import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain.chains import SequentialChain, LLMChain


from utils import mLangChain

In [4]:
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

llmchat = lcai.AzureChatOpenAI(
    openai_api_key=os.getenv("AZURE_OPENAI_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment="NUHAI-GPT4",
    openai_api_version="2024-02-15-preview",
    model_name="gpt-4",
)

embeddings = lcai.AzureOpenAIEmbeddings(
    openai_api_key=os.getenv("AZURE_OPENAI_KEY"),
    openai_api_version="2024-02-15-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment="TEST-Embedding",
)

In [5]:
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
contextualize_q_chain = contextualize_q_prompt | llmchat | StrOutputParser()

In [6]:
from langchain.chains import SimpleSequentialChain, SequentialChain

qa_info_prompt = """
            Your role is to act like a CUSTOMER seeking support. \
            The user is a support representative. \
            Respond to the question as if you were the customer. \
            If the user is asking for a specific detail, respond with a believable answer.
        """
qa_info = ChatPromptTemplate.from_messages(
    [
        ("system", qa_info_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
rag_chain_info = (
        RunnablePassthrough.assign(
            context=contextualize_q_chain
        )
        | qa_info
        | llmchat
)

qa_uncivil_prompt = """
            Given a history of messages, where the AI is a customer and the user is a representative, rephrase the response to the representative's message.\
            
            Rephrase the customer response to sound UNCIVIL. \
            Do NOT reply to the question ONLY rephrase. \
            
            This is what UNCIVIL customers do:\
            - Address others in an unprofessional, disrespectful way-for example, talking down, using degrading remarks or tone of voice.\
            - Pay little or no attention to others’ opinions.\
            - Use intimidating or threatening verbal communication—yelling, repeated emotional outbursts, threats, berating or harsh tone of voice, repeatedly interrupting.\
            - Blaming others for things out of their control.\
            - Accusing others of incompetence or dismissing their expertise.\
        """
qa_uncivil = ChatPromptTemplate.from_messages(
    [
        ("system", qa_uncivil_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", '''
            Representative asked: {question}
            Customer responded: {input}
        '''),
    ]
)
chain_uncivil = (RunnablePassthrough.assign(
                    context=contextualize_q_chain
                )
                 | qa_uncivil
                 | llmchat
                 )

In [7]:
content = """I just checked into my room and found that it hasn't been cleaned properly. There are towels from the previous guest and the trash hasn't been emptied. 
This isn't what I expected from a well-rated hotel."""
customer_complaint = [HumanMessage(content=content)]
chat_history = []
chat_history.append(content)
ai_response = rag_chain_info.invoke({'chat_history': chat_history, 'question': customer_complaint})
chat_history.append(ai_response.content)
print("AI Response 1:", ai_response.content)

AI Response 1: I'm really sorry to hear that your room was not in the condition you expected when you checked in. This is certainly not the standard we aim to uphold. Could you please provide me with your room number and the name the reservation is under? We'll have our housekeeping team address this immediately, and I'll also look into what might have happened with our cleaning process. Additionally, we'd like to offer you a complimentary service to make up for this inconvenience. Would you prefer a room upgrade, a free meal at our restaurant, or perhaps something else? Please let us know how we can make your stay more comfortable.


In [88]:
content = """How soon can I expect someone to clean it? You know what, I want a different room. Completely disgusted."""
customer_followup = [HumanMessage(content=content)]
chat_history.append(content)
ai_response_2 = rag_chain_info.invoke({'chat_history': chat_history, 'question': customer_followup})
chat_history.append(ai_response_2.content)
print("AI Response 2:", ai_response_2.content)

AI Response 2: I completely understand your frustration and apologize for the inconvenience. Let me arrange a different room for you right away. Could you please provide me with your current room number so I can expedite this process? We'll ensure the new room is thoroughly inspected and meets our high standards before you move in. I'll also make sure this is handled immediately.
